<center>
<h1><b>Homework 6</b></h1>
<h2>PHYS 580 - Computational Physics</h2>
<h4>Professor Molnar</h4>
</br>
<h5><b>Ethan Knox</b></h5>
<h6>https://www.github.com/ethank5149</h6>
<h6>ethank5149@gmail.com</h6>
</br>
<h5><b>December 4, 2020</b></h5>
</center>
<hr>

```mermaid

flowchart TB
    c1-->Initialize
    subgraph main.py
    InputParameters-->Initialize
    end
    subgraph util.py
    b1-->b2
    end
    subgraph wrapper.py
    c1-->c2
    end
    main.py --> util.py
    wrapper.py --> util.py
    util.py --> c2
```    

```mermaid 
classDiagram
    Rectangle <|-- Square
    class Rectangle~Shape~{
    int id
    List~string~ messages
    List~int~ position
    setMessages(List~string~ messages)
    setPoints(List~int~ points)
    getMessages() List~string~
    getPoints() List~int~
    }
```

### Imports

In [36]:
import warnings
from pylj import md, mc, util
import numpy as np
from numpy import pi, exp, sqrt
import matplotlib.pyplot as plt
from scipy.stats import chisquare, chi
from scipy.integrate import solve_ivp
from scipy.optimize import root_scalar
from tqdm.notebook import trange
from functools import partial
from matplotlib.animation import ArtistAnimation

warnings.filterwarnings('ignore')
%matplotlib widget

### Globals

# Problem 1
## 9.1 (p.283)

Calculate the speed distributions for a dilute gas as in Figure 9.4 and compare the results quantitatively with the Maxwell distribution. (For example, perform the $\chi^2$ analysis described in Appendix G.) This analysis also yields the temperature; compare the value you find with the result calculated directly from the equipartition theorem, $$k_BT=\left<\frac{m}{2}\left(v_x^2+v_y^2\right)\right>.$$

In [2]:
amu = 1.674e-27
kB  = 1.3806e-23
m   = 39.948 * amu

box_length = 12
number_of_steps = 5000
T = 273.15

alpha = sqrt(kB * T / m)
dist = chi(df=2, scale=alpha)
number_of_particles = box_length - 1 
speeds = np.zeros((number_of_steps, number_of_particles))

In [3]:
system = md.initialize(number_of_particles, T, box_length, 'square')
system.time = 0

In [4]:
for _ in trange(number_of_steps, desc='Sampling'):
    system.integrate(md.velocity_verlet)
    system.md_sample()
    system.heat_bath(T)
    system.time += system.timestep_length
    system.step += 1
    speeds[_] = sqrt(np.square(system.particles['xvelocity']) + np.square(system.particles['yvelocity']))
speeds = speeds.ravel()

In [5]:
alpha = np.sqrt(kB * T / m)
dist = chi(df=2, scale=alpha)  # Our simulation is 2D, and is therefore only 2 dof instead of the traditional 3
support = np.linspace(*dist.interval(0.999), 1000)

In [6]:
obs_per_bin = 1
df = number_of_particles // obs_per_bin
N = number_of_steps * number_of_particles - df

In [7]:
observed, bins = np.histogram(speeds, df)
expected = number_of_particles * number_of_steps * (dist.cdf(bins[1:]) - dist.cdf(bins[:-1]))
res = chisquare(observed, expected)

In [8]:
if 'e' in str(res[1]):
    a_str, b_str = str(res[1]).split('e')
    a, b = float(a_str), int(b_str)
    stringp = rf'${a:0.4g}\cdot10^{{{b}}}$'
else:
    stringp = rf'${res[1]:0.4g}$'

In [9]:
%%capture
fig, ax = plt.subplots(1,1,figsize=(16,9),dpi=300)
ax.set_title('Dilute Gas Simulation (Lennard-Jones Potential)')
ax.set_ylabel(r"$pdf$")
ax.set_xlabel(r"Speed $[\frac{m}{s}]$")

ax.plot(support, dist.pdf(support), label=rf'$y\sim\chi\left(2,\,\sqrt{{\frac{{k_BT}}{{m}}}}\right),\,\,T={T}\,[K]$')
ax.hist(speeds, bins=df, density=True, alpha=0.5, label=rf"$\chi^2\left({df},N={N}\right)={res[0]:0.4g},\,p=$" + stringp)
ax.legend()
ax.grid()
plt.savefig('Problem1.png')

![](./Problem1.png)

# Problem 2
## 10.5 (p.322)

Use the shooting method to study how the wave function for a particle-in-a-box depends on the magnitude of the potential outside the box $V_0$. Examine the variation of $\psi$ beyond the walls of the box and show that it decays exponentially with distance in this region. Study the decay length as a function of $V_0$ and compare the results for different energy levels. As the energy of the level approaches $V_0$ the decay length should become larger.

In [28]:
m = hbar = L = 1

In [29]:
def energy(n):
    return (n * pi * hbar / L) ** 2 / (2 * m)

def V(x, V0=1):
    return 0 if abs(x) < 0.5 * L else V0 

def TISE(x, psi, E, V0=1):
    return np.asarray([psi[1], (2 * m / hbar ** 2) * (V(x, V0) - E) * psi[0]])

In [30]:
def plot_wavefunction(E, xbound, psi0, V0):
    
    return fig, ax

In [69]:
def obj_func(E, psi0, xbound, V0):
    sol = solve_ivp(partial(TISE, E=E, V0=V0), t_span=xbound, y0=psi0)
    return sol.y[0,-1]

def finite_square_shoot(Ebound, xbound, psi0, V0):
    func = partial(obj_func, psi0=psi0, xbound=xbound, V0=V0)
    res = root_scalar(func, bracket=Ebound, method='brent')

    return res.root

In [73]:
xbound = [-L, L]
Ebound = [energy(1), energy(3)]
psi0 = [0., 1.]
Vmax = 100
t_eval = np.linspace(*xbound, 1000)
fig, ax = plt.subplots(1,1)
ax.set_ylim([-1.0, 1.])

artists = []

for i, v0 in enumerate(np.linspace(0.0, Vmax, 100)):
    try:
        E = finite_square_shoot(Ebound, xbound, psi0, v0)
        sol = solve_ivp(partial(TISE, E=E, V0=v0), xbound, psi0, t_eval=t_eval)
        artists.append(ax.plot(sol.t, sol.y[0,:]/np.max(sol.y[0,:]), c='b', label=f'E={E}'))
    except ValueError:
        print(f'V0 = {v0} not bracketed')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

V0 = 0.0 not bracketed
V0 = 1.0101010101010102 not bracketed
V0 = 2.0202020202020203 not bracketed
V0 = 3.0303030303030303 not bracketed
V0 = 4.040404040404041 not bracketed
V0 = 5.050505050505051 not bracketed
V0 = 6.0606060606060606 not bracketed
V0 = 7.070707070707071 not bracketed
V0 = 8.080808080808081 not bracketed
V0 = 9.090909090909092 not bracketed
V0 = 10.101010101010102 not bracketed
V0 = 11.111111111111112 not bracketed
V0 = 12.121212121212121 not bracketed
V0 = 13.131313131313131 not bracketed
V0 = 14.141414141414142 not bracketed
V0 = 15.151515151515152 not bracketed
V0 = 16.161616161616163 not bracketed
V0 = 17.171717171717173 not bracketed
V0 = 18.181818181818183 not bracketed
V0 = 19.191919191919194 not bracketed
V0 = 20.202020202020204 not bracketed
V0 = 21.212121212121215 not bracketed
V0 = 22.222222222222225 not bracketed
V0 = 23.232323232323235 not bracketed
V0 = 24.242424242424242 not bracketed
V0 = 25.252525252525253 not bracketed
V0 = 26.262626262626263 not brac

In [74]:
anim = ArtistAnimation(fig, artists)
anim.save('Problem2.mov')

# Problem 3
## 10.12 (p.333)

Employ the variational Monte Carlo method to calculate the ground-state energy and wave function of the anharmonic oscillator whose potential is given by $V(x)=x^4$.

# Problem 4
Write a matching method program to study the coupling between two one-dimensional quan-tum mechanical systems in neighboring square wells that areseparated by a small, square barrier(cf. Figs. 10.11 and 10.12 of the textbook). In particular, observe how identicalunperturbedstates in each well get mixed due to being coupled through thefinite barrier. Demonstratenumerically, for at at least two different examples (such as the two ground states and then twoexcited states), that the initially equal energy levels split up. Namely, the parity even mixturemoves down in energy, while the parity odd one moves up. This phenomenon is discussed inChapter 10 of the book (p.318-320)